In [18]:
%load_ext blackcellmagic
import pandas as pd
import numpy as np

import sqlite3 as sql
import datetime

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [21]:
connection = sql.connect("testy.db")

# Cursor used to traverse the records (?)
cursor = connection.cursor()

# cursor.execute("""DROP TABLE words;""")

In [33]:
connection = sql.connect("testy.db")

# Cursor used to traverse the records (?)
cursor = connection.cursor()

cursor.execute("""DROP TABLE words;""")

sql_command = """
CREATE TABLE words ( 
id INTEGER PRIMARY KEY, 
english VARCHAR(60), 
kana VARCHAR(60),
kanji VARCHAR(60),
week_id INT, 
date_added DATE);"""

cursor.execute(sql_command)
connection.commit()

In [34]:
# delete
# cursor.execute("""DROP TABLE employee;""")


# staff_data = [
#     ("William", "Shakespeare", "m", "1961-10-25"),
#     ("Frank", "Schiller", "m", "1955-08-17"),
#     ("Jane", "Wall", "f", "1989-03-14"),
# ]
WEEK_ID = 1
format_str = """
INSERT INTO words (
    id, english, kana, kanji, week_id, date_added
)
VALUES (
    NULL, "{english}", "{kana}", "{kanji}", "{week_id}", "{date_added}"
)
;"""
    
# DATE = str(datetime.date.today())
DATE = "2019-07-01"
for p in pairs[:5]:
    sql_command = format_str.format(
        english=p.english, kana="何",
        kanji=p.japanese,
        week_id=WEEK_ID,
        date_added=DATE)
    cursor.execute(sql_command)
    
DATE = "2019-06-27"
for p in pairs[5:10]:
    sql_command = format_str.format(
        english=p.english, kana="何",
        kanji=p.japanese,
        week_id=WEEK_ID,
        date_added=DATE)
    cursor.execute(sql_command)

# never forget this, if you want the changes to be saved:
connection.commit()

# necessary ?
# connection.close()

In [36]:
sql_command = """
SELECT * FROM words
WHERE date('now','-3 day') < date_added;"""
cursor.execute(sql_command)
result = cursor.fetchall() 
for r in result:
    print(r)

(1, 'Healthy', '何', 'けんこう', 1, '2019-07-01')
(2, 'Calculate', '何', 'けいさんする、計算する', 1, '2019-07-01')
(3, 'Precise', '何', 'せいかく、正確', 1, '2019-07-01')
(4, 'Trick someone', '何', 'だます、騙す', 1, '2019-07-01')
(5, 'Free', '何', 'むりょう, 無料', 1, '2019-07-01')


In [25]:
sql_command = """SELECT * FROM words;"""
cursor.execute(sql_command)
result = cursor.fetchall() 
for r in result:
    print(r)

(1, 'Healthy', '何', 'けんこう', 1, '2019-07-01')
(2, 'Calculate', '何', 'けいさんする、計算する', 1, '2019-07-01')
(3, 'Precise', '何', 'せいかく、正確', 1, '2019-07-01')
(4, 'Trick someone', '何', 'だます、騙す', 1, '2019-07-01')
(5, 'Free', '何', 'むりょう, 無料', 1, '2019-07-01')


Take X random
`SELECT * FROM table ORDER BY RANDOM() LIMIT X;`

* Filter words from today < 7 days
* Order randomly
* Pick X random from previous weeks
* Host somewhere (like https://dbhub.io/ ?), or simply just github/gitlab push

Perso:
* Create training mode


In [9]:
%load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

@dataclass
class Pairs:
    japanese: str
    english: str
    

@dataclass
class Player:
    name: str
    vocab: List[Pairs] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))


The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [10]:
_raw = """\
Healthy - けんこう
Calculate - けいさんする、計算する
Precise - せいかく、正確
Trick someone - だます、騙す
Free - むりょう, 無料
Trustable - しんよう (する), 信用【する】
A business - えいぎょう, 営業
As one would expect (/great) - さすが, 流石
International - こくさい, 国際
To entrust (to someone) - まかせる, 任せる
honest - しょうじき - 正直
feature - とくちょう - 特徴
basic - きほんてき - 基本的
to change - かえる - 変える
experiment - じっけん - 実験
efficiency - のうりつ - 能率
to improve - かいぜんする - 改善する
extreme - かげき - 過激
summary - おおすじ - 大筋
address - じゅうしょ - 住所
absence / non attendance - 欠席 / けっせき
rarely - めったに
to like / be pleased with - 気に入る / きにいる
grapes - ぶどう
sound / ring / roar - 鳴る / なる
board (board of wood) - 板 / いた
composition / essay - さくぶん　作文
surely certainly - きっと
to hold (hold the door) - 押さえる / おさえる
make a proposal / apply - 申し込む もうしこむ"""

pairs = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw.split("\n")]
]
# pairs

In [11]:
pairs

[Pairs(japanese='けんこう', english='Healthy'),
 Pairs(japanese='けいさんする、計算する', english='Calculate'),
 Pairs(japanese='せいかく、正確', english='Precise'),
 Pairs(japanese='だます、騙す', english='Trick someone'),
 Pairs(japanese='むりょう, 無料', english='Free'),
 Pairs(japanese='しんよう (する), 信用【する】', english='Trustable'),
 Pairs(japanese='えいぎょう, 営業', english='A business'),
 Pairs(japanese='さすが, 流石', english='As one would expect (/great)'),
 Pairs(japanese='こくさい, 国際', english='International'),
 Pairs(japanese='まかせる, 任せる', english='To entrust (to someone)'),
 Pairs(japanese='しょうじき - 正直', english='honest'),
 Pairs(japanese='とくちょう - 特徴', english='feature'),
 Pairs(japanese='きほんてき - 基本的', english='basic'),
 Pairs(japanese='かえる - 変える', english='to change'),
 Pairs(japanese='じっけん - 実験', english='experiment'),
 Pairs(japanese='のうりつ - 能率', english='efficiency'),
 Pairs(japanese='かいぜんする - 改善する', english='to improve'),
 Pairs(japanese='かげき - 過激', english='extreme'),
 Pairs(japanese='おおすじ - 大筋', english='summary'),
 Pair